In [11]:
import numpy as np
import matplotlib.pyplot as plt


with open("data_inputs/day10_input.txt") as f:
    data = f.read()

# ---- Part 1 ----

map_pipes = {
    ".": "0", 
    "|": "1", 
    "-": "2", 
    "L": "3", 
    "J": "4", 
    "7": "5", 
    "F": "6", 
    "S": "7"
}

for k, v in map_pipes.items():
    data = data.replace(k, v)

map_pipes = np.array([[int(n) for n in line] for line in data.split("\n")], dtype=np.int8)

map_pipes, map_pipes.shape

(array([[6, 3, 5, ..., 6, 6, 2],
        [6, 2, 5, ..., 6, 4, 4],
        [3, 6, 4, ..., 5, 2, 2],
        ...,
        [1, 5, 3, ..., 4, 0, 1],
        [2, 2, 5, ..., 3, 6, 1],
        [1, 4, 1, ..., 0, 2, 3]], dtype=int8),
 (140, 140))

In [25]:
check_connected[map_pipes[4, 5]]

{(-1, 0): [1, 5, 6], (1, 0): [1, 3, 4]}

In [32]:
pos

array([79, 85], dtype=int64)

In [31]:
map_pipes[pos]

array([[3, 0, 3, 2, 5, 3, 5, 5, 3, 1, 3, 2, 2, 4, 1, 6, 2, 4, 1, 6, 4, 1,
        5, 3, 4, 3, 2, 5, 6, 2, 5, 1, 3, 4, 2, 3, 4, 3, 5, 6, 6, 5, 6, 5,
        6, 2, 2, 4, 3, 4, 3, 4, 3, 4, 6, 2, 2, 2, 2, 5, 6, 2, 4, 1, 1, 6,
        5, 3, 6, 4, 6, 4, 0, 5, 1, 6, 5, 3, 2, 5, 1, 1, 1, 1, 6, 7, 1, 1,
        1, 1, 1, 1, 3, 5, 1, 6, 2, 4, 6, 5, 6, 5, 1, 6, 2, 2, 2, 4, 5, 6,
        4, 6, 2, 5, 6, 5, 6, 4, 3, 4, 1, 1, 4, 3, 4, 1, 3, 2, 2, 5, 3, 4,
        3, 4, 3, 4, 0, 5, 4, 5],
       [3, 6, 2, 4, 4, 4, 0, 6, 6, 5, 5, 6, 3, 4, 3, 3, 2, 5, 3, 4, 3, 4,
        3, 2, 4, 3, 5, 1, 6, 2, 4, 3, 2, 2, 4, 6, 5, 6, 2, 4, 2, 6, 5, 1,
        1, 6, 2, 2, 5, 3, 5, 1, 1, 3, 4, 1, 6, 2, 4, 6, 2, 5, 6, 5, 3, 5,
        3, 5, 1, 1, 6, 4, 3, 4, 1, 1, 1, 6, 4, 3, 5, 3, 5, 1, 6, 2, 2, 5,
        5, 6, 4, 6, 5, 3, 4, 3, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 6, 5,
        6, 5, 3, 2, 2, 4, 3, 4, 2, 6, 5, 0, 3, 4, 5, 6, 5, 4, 3, 3, 1, 5,
        0, 4, 4, 3, 0, 6, 4, 1]], dtype=int8)

In [30]:
check_connected = {
    0: None,
    1: {(-1, 0): [1, 5, 6], (1, 0): [1, 3, 4]},
    2: {(0, -1): [2, 3, 6], (0, 1): [2, 4, 5]},
    3: {(-1, 0): [1, 5, 6], (0, 1): [2, 4, 5]},
    4: {(-1, 0): [1, 5, 6], (0, -1): [2, 3, 6]},
    5: {(0, 1): [2, 3, 6], (-1, 0): [1, 5, 6]},
    6: {(1, 0): [1, 3, 4], (0, 1): [2, 3, 6]},
    7: {(-1, 0): [1, 5, 6], (1, 0): [1, 3, 4], (0, -1): [2, 3, 6], (0, 1): [2, 4, 5]}
}

map_connected = np.ones(map_pipes.shape, dtype=np.int8) * -1

start_pos = np.where(map_pipes == 7)
start_pos = (start_pos[0][0], start_pos[1][0])

pos = start_pos
i = 1

while pos != start_pos or (0 < i < map_pipes.shape[0] * map_pipes.shape[1]):
    pos = np.array(pos)
    check_dirs = check_connected[map_pipes[pos]]
    for dir in check_dirs:
        dir = np.array(dir)
        if map_pipes[pos + dir] in check_dirs[dir]:
            map_connected[pos + dir] = 1
            pos += dir
            break
        else:
            map_connected[pos + dir] = 0
    
    i += 1

plt.imshow(map_connected)

TypeError: unhashable type: 'numpy.ndarray'

In [29]:
i

1